<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/HUSE_per_pacient/Gower3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Taula amb dades huse

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data1 = pd.read_csv('df_gower.csv')

In [ ]:
data1.head()

,ID,CLAVE,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,222,222,0,1,82,47,64.0,1.47,29.62,3.0,...,0,0,0,0,0,0,0,0,0,1
1,346,346,0,1,71,26,55.0,1.52,23.81,2.0,...,0,0,0,0,0,0,0,0,0,1
2,613,613,1,0,48,75,65.0,1.69,22.76,2.0,...,2,0,0,0,1,0,0,1,1,1
3,697,697,1,0,77,110,68.0,1.59,26.90,3.0,...,0,0,0,0,0,0,0,0,1,0
4,745,745,0,0,54,28,65.0,1.70,22.49,2.0,...,2,0,1,1,1,1,1,0,1,1


In [ ]:
data1.drop(['ID', 'CLAVE'], axis=1)

,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,HCV,Alcohol _abuse\n,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,0,1,82,47,64.0,1.47,29.62,3.0,1.0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,71,26,55.0,1.52,23.81,2.0,1.0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,48,75,65.0,1.69,22.76,2.0,1.0,1,...,2,0,0,0,1,0,0,1,1,1
3,1,0,77,110,68.0,1.59,26.90,3.0,1.0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,54,28,65.0,1.70,22.49,2.0,1.0,0,...,2,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1,1,83,194,75.0,1.60,29.30,3.0,1.0,0,...,2,0,0,0,1,0,0,0,1,1
127,1,0,67,578,94.0,1.74,31.05,4.0,1.0,1,...,0,0,0,0,0,0,0,0,0,1
128,1,0,67,52,65.0,1.74,21.47,2.0,0.0,1,...,1,0,0,0,0,0,0,1,1,0
129,1,0,74,269,83.0,1.65,30.49,4.0,0.0,1,...,2,0,0,1,1,0,0,0,1,1


In [ ]:
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut

data1.columns = data1.columns.str.strip()

y = data1.loc[:, "RECIST"].values

X = data1.loc[:, (data1.columns != "RECIST") & (data1.columns != "ID")].values

#Escalam X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Cream la regressió logística amb la penalització lasso (l1)
loo = LeaveOneOut()
model = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

all_bootstrap_probabilities = []

# Aplicar LOOCV amb Bootstrap
for train_index, test_index in loo.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    bootstrapped_probabilities = []

    # Generar mostres i fer prediccions
    for _ in range(1000):  # Generar 1000 mostres bootstrap
        X_train_bootstrap, y_train_bootstrap = resample(X_train, y_train, replace=True)

        model.fit(X_train_bootstrap, y_train_bootstrap)

        prob = model.predict_proba(X_test)[:, 1][0]
        bootstrapped_probabilities.append(prob)

    all_bootstrap_probabilities.append(bootstrapped_probabilities)

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

df_probabilities.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.981532,0.143217,0.230682,0.974761,0.912271,0.996570,0.596957,0.896327,0.540873,0.987779,...,0.972160,0.986860,0.957118,0.553557,0.930839,0.780611,0.960285,0.953930,0.892973,0.901995
1,0.748715,0.955674,0.906421,0.936452,0.719208,0.959102,0.985515,0.886872,0.922614,0.967757,...,0.941200,0.929191,0.886018,0.923449,0.976573,0.844496,0.900757,0.877440,0.524044,0.930822
2,0.271399,0.980761,0.852042,0.189528,0.775817,0.342277,0.616091,0.833592,0.057028,0.975136,...,0.949876,0.950027,0.905411,0.724968,0.395517,0.558835,0.623313,0.257855,0.802756,0.779784
3,0.979905,0.918900,0.825472,0.912393,0.981910,0.922528,0.929479,0.963452,0.994703,0.912105,...,0.900244,0.940434,0.959832,0.995690,0.898747,0.953084,0.937459,0.956037,0.674213,0.973541
4,0.621219,0.915426,0.997349,0.907222,0.993154,0.999113,0.908736,0.992137,0.982040,0.675256,...,0.780470,0.999682,0.999038,0.184830,0.996618,0.992393,0.995726,0.945071,0.800604,0.346891


In [ ]:

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

data1 = pd.read_csv('df_gower.csv')
data1.drop(['CLAVE'],axis=1,inplace=True)
df_probabilities.insert(0, 'ID', data1['ID'].values)
df_probabilities.to_csv("huse5bootstrap_probabilities_id.csv", index=False)



print("Archivo guardado con el identificador 'id' en la primera columna.")

Archivo guardado con el identificador 'id' en la primera columna.


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [ ]:

taula1 = pd.read_csv('huse5bootstrap_probabilities_id.csv')
taula2 = pd.read_csv('df_gower.csv')

taula2.drop(['CLAVE'],axis=1,inplace=True)

In [ ]:

id_column = taula1.iloc[:, 0]

probabilities = taula1.iloc[:, 1:]

y_pred = (probabilities.mean(axis=1) >= 0.5).astype(int)

taula1_processed = pd.DataFrame({'ID': id_column, 'RECIST': y_pred})

taula2_processed = taula2[['ID', 'RECIST']]
taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)

taula05 = pd.merge(taula1_processed, taula2_processed, on='ID')

print(taula05.head())

    ID  RECIST  y_true
0  222       1       0
1  346       1       0
2  613       1       1
3  697       1       1
4  745       1       0


<ipython-input-10-d60078a473e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)


In [ ]:
y_pred = taula05['RECIST']
y_true = taula05['y_true']

cm = confusion_matrix(y_true, y_pred)
print("Matriu de confusió:")
print(cm)

precision = precision_score(y_true, y_pred)
print(f"Precisió: {precision:.2f}")

recall = recall_score(y_true, y_pred)
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_true, y_pred)
print(f"F1-Score: {f1:.2f}")

accuracy = accuracy_score(y_true, y_pred)
print(f"Exactitut: {accuracy:.2f}")

Matriu de confusió:
[[  0  21]
 [  9 101]]
Precisió: 0.83
Recall: 0.92
F1-Score: 0.87
Exactitut: 0.77


Veim que les dades estan molt desbalancetjades.

In [ ]:
df_data = pd.read_csv("df_gower.csv")
df_data.drop(['CLAVE'],axis=1,inplace=True)

df_probabilities = pd.read_csv("huse5bootstrap_probabilities_id.csv")

df_data_subset = df_data[['ID', 'RECIST']]

if len(df_data_subset) == len(df_probabilities):
    df_final = pd.concat([df_data_subset, df_probabilities], axis=1)

    df_final.to_csv("final_table.csv", index=False)

    print("Tabla final creada y guardada en 'final_table.csv'.")
else:
    print("Error: Los DataFrames tienen diferente número de filas. Verifica tus datos.")

Tabla final creada y guardada en 'final_table.csv'.


In [ ]:
df = pd.read_csv('final_table.csv')

id_column = 'ID'
diagnosis_column = 'RECIST'
prob_columns = df.columns[3:]

def calcular_intervalo(probabilidades, li, ls):
    lower_bound = np.percentile(probabilidades, li)
    upper_bound = np.percentile(probabilidades, ls)
    return lower_bound, upper_bound

In [ ]:
intervalos_confianza = []

for index, row in df.iterrows():
    probabilidades = row[prob_columns].values  # Extraer las probabilidades
    lower, upper = calcular_intervalo(probabilidades, 2.5, 97.5)  # Calcular el intervalo de confianza
    intervalos_confianza.append([row[id_column], row[diagnosis_column], lower, upper])

# Crear un nuevo DataFrame con los intervalos de confianza
df_intervalos = pd.DataFrame(intervalos_confianza, columns=[id_column, diagnosis_column, 'lower_bound', 'upper_bound'])

In [ ]:
def calcular_TP(a, b):
    TP = min(1 - a, b)
    return TP
def calcular_AP(a,b):
    AP = min(a, 1-b)
    return AP
def calcular_EP(a,b):
    EP = (b-a)
    return EP

In [ ]:
# Añadir la columna TP aplicando la función calcular_TP a lower_bound y upper_bound
df_intervalos['TP'] = df_intervalos.apply(lambda row: calcular_TP(row['lower_bound'], row['upper_bound']), axis=1)

# Añadir la columna AP aplicando la función calcular_AP a lower_bound y upper_bound
df_intervalos['AP'] = df_intervalos.apply(lambda row: calcular_AP(row['lower_bound'], row['upper_bound']), axis=1)

# Añadir la columna EP aplicando la función calcular_EP a lower_bound y upper_bound
df_intervalos['EP'] = df_intervalos.apply(lambda row: calcular_EP(row['lower_bound'], row['upper_bound']), axis=1)

In [ ]:
# Calcular medias de TP, AP y EP
medias = df_intervalos[['TP', 'AP', 'EP']].mean()
print("Medias:")
print(medias)

Medias:
TP    0.665010
AP    0.002638
EP    0.662372
dtype: float64
